In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.model = nn.Sequential(
            # 1st layer
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 256 / 2 = 128

            # 2nd layer
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 128 / 2 = 64

            # 3rd layer
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size= 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 64 / 2 = 32

            # 4rd layer
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size= 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 32 / 2 = 16

            nn.Flatten(), #256 * 16 * 16

            # 1st linear
            nn.Linear(256 * 16 * 16, 512),
            nn.ReLU(),
            nn.Dropout(0.5),

            # 2nd linear
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),

            # 3nd linear
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),

            # 4rd linear
            nn.Linear(64, 16),
            nn.ReLU(),

            # 5th linear
            nn.Linear(16, 4)
        )

    def forward(self, x):
        return self.model(x)

In [2]:
def train(model, train_loader, val_loader, optimiser, criterion, epochs, device, writer, ckpt_path="../models/best.pt"):
    model.to(device)

    best = 0.0

    for epoch in range(epochs):
        train_loop = (tqdm(enumerate(train_loader, 0), total= (len(train_loader)), desc=f"Epoch {epoch}"))  

        model.train()
        train_loss = 0.

        for batch_idx, (feat, labels) in train_loop:
            feat, labels = feat.to(device), labels.to(device)
            
            outputs = model(feat)

            loss = criterion(outputs, labels)

            optimiser.zero_grad()
            loss.backward()
            optimiser.step() 
            
            train_loss += loss.item()

            train_loop.set_postfix({"loss": loss.item()})
            
        if writer:
            writer.add_scalar("Loss/train", train_loss / len(train_loader), epoch)
        
        with torch.no_grad():
            correct, total = 0, 0
            model.eval()
            val_loop = tqdm(enumerate(val_loader, 0), total=len(val_loader), desc="Val")
            for batch_idx, (feat, labels) in val_loop:
                feat, labels = feat.to(device), labels.to(device)

                outputs = model(feat)
                _, preds = torch.max(outputs.data, 1)

                correct += (preds == labels).sum().item()
                total += labels.size(0)

                val_loop.set_postfix({"acc": correct / total})
            
            if writer:
                writer.add_scalar("Accuracy/val", correct / total, epoch)

            if correct / total > best:
                torch.save(model.state_dict(), ckpt_path)
                best = correct / total
    return train_loss, correct / total 
        

In [3]:
def select_device():
    if torch.backends.mps.is_available():
        return 'mps'
    elif torch.cuda.is_available():
        return 'cuda'
    else:
        return 'cpu'

In [5]:
from data import load_data

data_version = "v1.0.0"

train_dataset, val_dataset = load_data(data_version)

Dataset version v1.0.0 loaded! Train size: 2870, Class counts: {'0': 826, '1': 822, '2': 395, '3': 827}


/Users/maxmartyshov/Desktop/IU/year3/PMDL/PMDL_Brain_tumor_classification/code/versioning.py:148: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_dataset = torch.load(os

In [8]:
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from tqdm import tqdm

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

model = CNN()
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
device = select_device()
epochs = 10

train_loss, val_acc = train(model, train_loader, val_loader, optimiser, loss_fn, epochs, device, SummaryWriter())


Val: 100%|██████████| 4/4 [00:00<00:00, 14.13it/s, acc=0.589]


In [ ]:
from versioning import save_model, get_next_version

print(get_next_version(type="model"))

save_model(model, optimiser, epoch=epochs, loss=train_loss, accuracy=val_acc, version=get_next_version(type="model"))